In this work, you are required to build a GNN training pipline. Then you can truly use the Graph Neural Network.

In [1]:
# !pip install  dgl -f https://data.dgl.ai/wheels/repo.html
# !pip install torch_geometric
# !pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.1+cpu.html

First, we need to download the dataset and load data.

In [2]:
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
dataset = Planetoid("../", "Cora", transform=T.NormalizeFeatures())
data = dataset[0]

x = data.x
# edge_index = data.edge_index
# edge_weight = data.edge_weight

Then, you need to implement a GNN model. You may copy the GCNConv from your work two weeks ago, and build the model with the convolution layers.

In [3]:
from torch_geometric.nn import MessagePassing
import torch.nn as nn
import torch.nn.functional as F 
# class PyG_GCNConv(MessagePassing):
#   # Your code here
#   def __init__(self, in):
#       
#   # End code here
# 

class PyG_GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(PyG_GCNConv, self).__init__(aggr='add')  # 聚合函数为加法
        self.input_channel = in_channels
        self.output_channel = out_channels
        self.W = torch.nn.Parameter(torch.ones(in_channels, out_channels))
        self.bias = torch.nn.Parameter(torch.ones(out_channels))
        # torch.nn.init.xavier_uniform_(self.W)  # 使用Xavier初始化
        # torch.nn.init.zeros_(self.bias)

    def forward(self, x, edge_index,edge_weight):
        # 可选：为邻接矩阵添加自环
        # edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        
        # print("edge_index:", edge_index.shape)
        # print("x before linear:", x.shape)
        
        # 线性变换
        x = torch.matmul(x, self.W)
        
        # print("x after linear:", x.shape)
        
        # 开始消息传递
        out = self.propagate(edge_index, x=x,edge_weight=edge_weight)
        
        # 添加偏置
        out += self.bias
        return out

    # 重写message函数
    def message(self, x_j, edge_weight,edge_index):
        # print("x_j:", x_j.shape)
        if edge_weight is None:
            return x_j
        else:
            # x = x_j[edge_index[0,:]]
            # print("edge_weight:", edge_weight.shape)
            return edge_weight.view(-1, 1) * x_j

    # 如有需要，可以重写update函数（可选）
    def update(self, aggr_out):
        return aggr_out
class PyG_GCN(nn.Module):
  # Your code here
  def __init__(self,in_channels, out_channels, hidden_dim):
      super(PyG_GCN,self).__init__()
      self.conv1 = PyG_GCNConv(in_channels,hidden_dim)
      self.conv2 = PyG_GCNConv(hidden_dim, out_channels)
  def forward(self,data):
      #传入进数据
      x, edge_index = data.x, data.edge_index
      #经过每层网络
      x = self.conv1(x,data.edge_index,data.edge_weight)
      x = F.relu(x)
      x = F.dropout(x, training = self.training)
      x = self.conv2(x, data.edge_index,data.edge_weight)
      return F.log_softmax(x,dim=1)
  # End code here

Building the training and evaluation part, this is similar to the work in week4. Our downstream task is just node classification.

In [4]:
from torch_geometric.logging import log
import torch
# Build your training pipeline
hidden_dim = 16
lr = 0.001
epochs = 100
model = PyG_GCN(dataset.num_features, hidden_dim, dataset.num_classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
data = data.to(device)
optimizer = torch.optim.Adam(
    params=model.parameters(),
    lr=lr,
    weight_decay=5e-4
)

def train():
  # Your code here
      #梯度是累积的（即每次调用 backward() 时，计算出的梯度会被加到已有的梯度上）。如果不清零梯度，那么在每次更新参数时，梯度都会包含当前批次以及所有之前批次的梯度，这样会导致错误的参数更新。optimizer.zero_grad()用于在每次进行反向传播和更新模型参数之前，将所有模型参数的梯度清零。这样可以避免累积梯度，从而确保每次迭代时梯度计算都是基于当前的批次数据。
      optimizer.zero_grad()
      out = model(data)
      loss = F.nll_loss(out[data.train_mask],data.y[data.train_mask])
      #反向传播,计算梯度
      loss.backward()
      #用于根据上一步计算的梯度更新模型的参数，以最小化损失函数
      optimizer.step()
      return loss
  # End code here
# 用于在不需要计算梯度时，临时禁用梯度计算。提高计算效率
@torch.no_grad()
def test():
  model.eval()
  pred = model(data).argmax(dim=-1)

  accs = []
  for mask in [data.train_mask, data.val_mask, data.test_mask]:
      accs.append(int((pred[mask] == data.y[mask]).sum()) / int(mask.sum()))
  return accs
best_val_acc = 0
for epoch in range(1, epochs + 1):
  loss = train()
  train_acc, val_acc, tmp_test_acc = test()
  
  if val_acc > best_val_acc:
      best_val_acc = val_acc
      test_acc = tmp_test_acc
  log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)

Epoch: 001, Loss: 2.772589683532715, Train: 0.1429, Val: 0.1220, Test: 0.1300
Epoch: 002, Loss: 2.4910247325897217, Train: 0.1429, Val: 0.0720, Test: 0.1300
Epoch: 003, Loss: 2.366645097732544, Train: 0.1429, Val: 0.0720, Test: 0.1300
Epoch: 004, Loss: 2.2941508293151855, Train: 0.1429, Val: 0.1560, Test: 0.1440
Epoch: 005, Loss: 2.2474563121795654, Train: 0.1429, Val: 0.1560, Test: 0.1440
Epoch: 006, Loss: 2.2103235721588135, Train: 0.1429, Val: 0.1560, Test: 0.1440
Epoch: 007, Loss: 2.176870107650757, Train: 0.1429, Val: 0.1560, Test: 0.1440
Epoch: 008, Loss: 2.1495609283447266, Train: 0.1429, Val: 0.1560, Test: 0.1440
Epoch: 009, Loss: 2.130228281021118, Train: 0.1429, Val: 0.1580, Test: 0.1430
Epoch: 010, Loss: 2.1143524646759033, Train: 0.1357, Val: 0.1520, Test: 0.1430
Epoch: 011, Loss: 2.100473642349243, Train: 0.1429, Val: 0.1540, Test: 0.1430
Epoch: 012, Loss: 2.089570999145508, Train: 0.1429, Val: 0.1560, Test: 0.1430
Epoch: 013, Loss: 2.0795352458953857, Train: 0.1429, Val: 

Now, you can train the GCN model with PyG. Next, you may try using the DGL to implement the similiar function.

In [5]:
import argparse

import dgl
import dgl.nn as dglnn

import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl import AddSelfLoop
# data bug ,the reason is the 0cloud, the host is wrong, turn off it.
from dgl.data import CoraGraphDataset


transform = (
        AddSelfLoop()
    )
data = CoraGraphDataset(transform=transform)
g = data[0]

features = g.ndata["feat"]
labels = g.ndata["label"]
masks = g.ndata["train_mask"], g.ndata["val_mask"], g.ndata["test_mask"]
# edge_weight = g.ndata['edata']

class DGL_conv(nn.Module):
    def __init__(self, input_channels, output_channels):
        super(DGL_conv, self).__init__()
        self.input_channel = input_channels
        self.output_channel = output_channels
        self.W = nn.Parameter(torch.ones((input_channels,output_channels)))
        self.b = nn.Parameter(torch.ones(output_channels))
    def forward (self, g, h):
        with g.local_scope(): # local_scope 函数的目的是在这个区域进行信息传递，聚合和更新，但是并不修改原图的节点或边的特征
            # g.edata['w'] = edge_weight
            h = torch.matmul(h, self.W)
            # h = torch.matmul(h,g.edges['w'])
            g.ndata['h'] = h
            # g.update_all(dgl.function.u_mul_e('h','w', 'm'), dgl.function.sum('m', 'h_sum'))
            #如果没有边，则直接用节点的信息根据边的连接情况进行消息传递
            g.update_all(dgl.function.copy_u('h', 'm'), dgl.function.sum('m', 'h_sum'))
            h_sum = g.ndata['h_sum']
            h_message = h_sum + self.b
            return h_message

class DGL_GCN(nn.Module):
  # Your code here
  def __init__(self, in_channels, out_channels, hidden_dim):
      super(DGL_GCN,self).__init__()
      self.conv1 = DGL_conv(in_channels, hidden_dim)
      self.conv2 = DGL_conv(hidden_dim, out_channels)
  def forward(self,g, h):
  
      #经过每层网络
      x = self.conv1(g, h)
      x = F.relu(x)
      x = F.dropout(x, training = self.training)
      x = self.conv2(g, x)
      return F.log_softmax(x,dim=1)    
      
  # End code here

def train(g, features, labels, masks, model):
  # Your code here
  train_mask = masks[0]
  print('---------',train_mask,len(train_mask))
  
  val_mask = masks[1]
  print('---------',val_mask,len(val_mask))
  loss_fcn = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=5e-4)
  
  for epoch in range(200):
      model.train()
      logits = model(g,features)
      loss = loss_fcn(logits[train_mask],labels[train_mask])
      optimizer.zero_grad()
      # out = model(g, features)
      # loss = F.nll_loss(out[data.train_mask],data.y[data.train_mask])
      #反向传播,计算梯度
      loss.backward()
      #用于根据上一步计算的梯度更新模型的参数，以最小化损失函数
      optimizer.step()
      acc = evaluate(g, features, labels, val_mask, model)
      print(
            "Epoch {:05d} | Loss {:.4f} | Accuracy {:.4f} ".format(
                epoch, loss.item(), acc
            )
        )
  # End code here

def evaluate(g, features, labels, mask, model):
  model.eval()
  with torch.no_grad():
    logits = model(g, features)
    logits = logits[mask]
    labels = labels[mask]
    _, indices = torch.max(logits, dim=1)
    correct = torch.sum(indices == labels)
    return correct.item() * 1.0 / len(labels)

model = DGL_GCN(features.shape[1],data.num_classes, hidden_dim)
print("Training...")
train(g, features, labels, masks, model)

# test the model
print("Testing...")
acc = evaluate(g, features, labels, masks[2], model)
print("Test accuracy {:.4f}".format(acc))

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Training...
--------- tensor([ True,  True,  True,  ..., False, False, False]) 2708
--------- tensor([False, False, False,  ..., False, False, False]) 2708
Epoch 00000 | Loss 1.9459 | Accuracy 0.1560 
Epoch 00001 | Loss 5.3192 | Accuracy 0.0720 
Epoch 00002 | Loss 8.2273 | Accuracy 0.1220 
Epoch 00003 | Loss 11.9271 | Accuracy 0.1620 
Epoch 00004 | Loss 7.9609 | Accuracy 0.0580 
Epoch 00005 | Loss 4.9593 | Accuracy 0.1140 
Epoch 00006 | Loss 8.5841 | Accuracy 0.1560 
Epoch 00007 | Loss 7.0817 | Accuracy 0.1560 
Epoch 00008 | Loss 7.5332 | Accuracy 0.3160 
Epoch 00009 | Loss 5.2327 | Accuracy 0.0720 
Epoch 00010 | Loss 6.4476 | Accuracy 0.0720 
Epoch 00011 | Loss 7.2973 | Accuracy 0.0720 
Epoch 00012 | Loss 5.2788 | Accuracy 0.0720 
Epoch 00013 | Loss 5.2488 | Accuracy 0.1220 
Epoch 00014 | Loss 3.4337 | Acc

If you find it hard to implement, you may refer to the official implementation of the GNN training, like [PyG](https://github.com/pyg-team/pytorch_geometric/blob/master/examples/gcn.py) and [DGL](https://github.com/dmlc/dgl/blob/master/examples/pytorch/gcn/train.py).